In [1]:
import time
import datetime
import random
import pandas as pd 
import pickle
import json
import requests
from io import BytesIO
from itertools import repeat

from pytimekr import pytimekr
import yfinance as yf
import pykrx
import dart_fss
from fredapi import Fred
import talib as ta
import matplotlib.pyplot as plt
import mplfinance as mpf
import plotly.graph_objects as go

from yf_func import *
from krx_func import *
from utils import *

OPENDART_APIKEY = "f15d5e316baebf5fa004412c2feefb4a8dd65745"
dart_fss.set_api_key(api_key=OPENDART_APIKEY)

FRED_API_KEY = "89ac04db790be30df44979764e3aaf69"

# 나스닥

In [2]:
ticker = "TSM"
company = yf.Ticker(ticker)
info = company.info
income_statement = company.quarterly_income_stmt  # 손익계산서, is
balance_sheet = company.quarterly_balance_sheet   # 재무상태표(대차대조표), bs
cashflow = company.quarterly_cashflow             # 현금흐름표, cf

In [3]:
extractor = YahooFinanceInfoExtractor()
extractor.download_fs(ticker)

True

In [5]:
extractor.PEGR(ticker)

0.4113050007674862

In [7]:
eps_ttm = extractor.EPS(ticker, True)

In [8]:
np.mean([calc_growth_rate(v, eps_ttm[i+1]) for i, v in enumerate(eps_ttm[:-1])])

8.721821926147857

In [25]:
extractor.info

{'address1': 'Hsinchu Science Park',
 'address2': 'No. 8, Li-Hsin Road 6',
 'city': 'Hsinchu City',
 'zip': '300096',
 'country': 'Taiwan',
 'phone': '886 3 563 6688',
 'fax': '886 3 563 7000',
 'website': 'https://www.tsmc.com',
 'industry': 'Semiconductors',
 'industryKey': 'semiconductors',
 'industryDisp': 'Semiconductors',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Taiwan Semiconductor Manufacturing Company Limited, together with its subsidiaries, manufactures, packages, tests, and sells integrated circuits and other semiconductor devices in Taiwan, China, Europe, the Middle East, Africa, Japan, the United States, and internationally. It provides a range of wafer fabrication processes, including processes to manufacture complementary metal- oxide-semiconductor (CMOS) logic, mixed-signal, radio frequency, embedded memory, bipolar CMOS mixed-signal, and others. The company also offers customer and engineering support se

In [34]:
extractor.get_price(ticker) / extractor.EPS(ticker)

3.5873288707566724

In [33]:
extractor.PER(ticker)

3.5873289740281225

# 코스피

In [4]:
stock_code = '005930'
corp_code = '00126380'  # 삼성전자

In [7]:
corp_list = dart_fss.get_corp_list()
company = corp_list.find_by_stock_code(stock_code)

In [9]:
begin = datetime.date.today() - datetime.timedelta(days=730)
fs = company.extract_fs(bgn_de=begin.strftime("%Y%m%d"), report_tp="quarter")

Annual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/4 [00:00<?, ?report/s]

In [12]:
fs["is"].iloc[:, 1:3]

[D310000] 손익계산서, 기능별 분류 - 연결 | Income statement, by function of expense - Consolidated financial statements (Unit: KRW)  \
                                                                                                                  label_ko   
0                                                영업수익                                                                        
1                                                매출원가                                                                        
2                                               매출총이익                                                                        
3                                             판매비와관리비                                                                        
4                                                영업이익                                                                        
5                                                기타이익                                                                        
6                                                기타손실                                                                        
7                                               지분법이익                                                                        
8                                                금융수익                                                                        
9                                                금융비용                                                                        
10                                    법인세비용차감전순이익(손실)                                                                        
11                                          법인세비용(수익)                                                                        
12                                         계속영업이익(손실)                                                                        
13                                          당기순이익(손실)                                                                        
14                         지배기업의 소유주에게 귀속되는 당기순이익(손실)                                                                        
15                              비지배지분에 귀속되는 당기순이익(손실)                                                                        
16                                         기본주당이익(손실)                                                                        
17                                         희석주당이익(손실)                                                                        

                                                       
                                             label_en  
0                                             Revenue  
1                                       Cost of sales  
2                                        Gross profit  
3                 Selling and administrative expenses  
4                             Operating profit (loss)  
5                                         Other gains  
6                                        Other losses  
7   Share of net profit of associates and joint ve...  
8                                    Financial income  
9                                   Financial expense  
10                           Profit before income tax  
11                       Income tax expense (benefit)  
12           Profit (loss) from continuing operations  
13                                Profit for the year  
14  Profit attributable to Owners of the parent co...  
15   Profit attributable to Non-controlling interests  
16                                            - Basic  
17                                          - Diluted

In [13]:
fs["bs"].iloc[:, 1:3]

[D210000] 재무상태표, 유동/비유동법 - 연결 | Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW)  \
                                                                                                                               label_ko   
0                                                유동자산                                                                                     
1                                            현금및현금성자산                                                                                     
2                                              단기금융상품                                                                                     
3                                         단기상각후원가금융자산                                                                                     
4                                     단기당기손익-공정가치금융자산                                                                                     
5                                                매출채권                                                                                     
6                                                 미수금                                                                                     
7                                                선급비용                                                                                     
8                                                재고자산                                                                                     
9                                              기타유동자산                                                                                     
10                                           매각예정분류자산                                                                                     
11                                              비유동자산                                                                                     
12                             기타포괄손익-공정가치 측정 비유동금융자산                                                                                     
13                                      당기손익-공정가치금융자산                                                                                     
14                                     관계종속기업투자자산-지분법                                                                                     
15                                               유형자산                                                                                     
16                                               무형자산                                                                                     
17                                            순확정급여자산                                                                                     
18                                            이연법인세자산                                                                                     
19                                            기타비유동자산                                                                                     
20                                               자산총계                                                                                     
21                                               유동부채                                                                                     
22                                               매입채무                                                                                     
23                                              단기차입금                                                                                     
24                                               미지급금                                                                                     
25                                                선수금                                                                                     
26                                                예수금                                                          

In [14]:
fs["cf"].iloc[:, 1:3]

[D520000] 현금흐름표, 간접법 - 연결 | Statement of cash flows, indirect method - Consolidated financial statements (Unit: KRW)  \
                                                                                                               label_ko   
0                                            영업활동현금흐름                                                                     
1                                       영업에서 창출된 현금흐름                                                                     
2                                               당기순이익                                                                     
3                                                  조정                                                                     
4                                  영업활동으로 인한 자산부채의 변동                                                                     
5                                              이자의 수취                                                                     
6                                              이자의 지급                                                                     
7                                              배당금 수입                                                                     
8                                             법인세 납부액                                                                     
9                                            투자활동현금흐름                                                                     
10                                    단기금융상품의 순감소(증가)                                                                     
11                               단기상각후원가금융자산의 순감소(증가)                                                                     
12                           단기당기손익-공정가치금융자산의 순감소(증가)                                                                     
13                                         장기금융상품의 처분                                                                     
14                                         장기금융상품의 취득                                                                     
15                               기타포괄손익-공정가치측정금융자산의처분                                                                     
16                               기타포괄손익-공정가치측정금융자산의취득                                                                     
17                                  당기손익-공정가치금융자산의 처분                                                                     
18                                  당기손익-공정가치금융자산의 취득                                                                     
19                                 관계기업 및 공동기업 투자의 처분                                                                     
20                                 관계기업 및 공동기업 투자의 취득                                                                     
21                                           유형자산의 처분                                                                     
22                                           유형자산의 취득                                                                     
23                                           무형자산의 처분                                                                     
24                                           무형자산의 취득                                                                     
25                                    사업결합으로 인한 현금유출액                                                                     
26                      매각예정으로 분류된 비유동자산이나 처분자산집단의 처분                                                                     
27                                 기타투자활동으로 인한 현금유출입액                                                                     
28                                           재무활동현금흐름                                                                     
29                                     단기차입금의 순증가(감소)                                                                     
30                                          장기차입금의 차입              

In [3]:
extractor = DartInfoExtractor()

In [11]:
extractor.earning(stock_code, seperate=True)

9737437000000.0

In [12]:
manager.earning(stock_code, seperate=False)

6621030000000.0

In [9]:
manager.equity(stock_code, seperate=True)

232139453000000.0

In [10]:
manager.equity(stock_code, seperate=False)

371916124000000.0

In [13]:
manager.ROE(stock_code)

4.194649756497876

In [ ]:
66,210

# dart_fss

In [10]:
def get_fs_df(fs, fs_type, row_name):
    assert fs_type in ["bs", "is", "cf"]    # [재무상태표, 손익계산서, 현금흐름표]
    df = fs[fs_type].copy()     
    df.columns = df.columns.droplevel(level=1)
    if fs_type == "bs":
        df = df[df.iloc[:, 1] == row_name].filter(regex="^[0-9]+$")
    else:
        df = df[df.iloc[:, 1] == row_name].filter(regex="^[0-9]+-[0-9]+$")
    return df

In [11]:
corp_list = dart_fss.get_corp_list()
company = corp_list.find_by_stock_code("005930")

In [12]:
begin = datetime.date.today() - datetime.timedelta(days=730)
fs = company.extract_fs(bgn_de=begin.strftime("%Y%m%d"), report_tp="quarter")

Annual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/4 [00:00<?, ?report/s]

In [13]:
df_bs = fs['bs'].copy()
df_is = fs['is'].copy()
df_cf = fs['cf'].copy()

In [14]:
df_bs.iloc[:, [1, 2, 8]]

[D210000] 재무상태표, 유동/비유동법 - 연결 | Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW)  \
                                                                                                                               label_ko   
0                                                유동자산                                                                                     
1                                            현금및현금성자산                                                                                     
2                                              단기금융상품                                                                                     
3                                         단기상각후원가금융자산                                                                                     
4                                     단기당기손익-공정가치금융자산                                                                                     
5                                                매출채권                                                                                     
6                                                 미수금                                                                                     
7                                                선급비용                                                                                     
8                                                재고자산                                                                                     
9                                              기타유동자산                                                                                     
10                                           매각예정분류자산                                                                                     
11                                              비유동자산                                                                                     
12                             기타포괄손익-공정가치 측정 비유동금융자산                                                                                     
13                                      당기손익-공정가치금융자산                                                                                     
14                                     관계종속기업투자자산-지분법                                                                                     
15                                               유형자산                                                                                     
16                                               무형자산                                                                                     
17                                            순확정급여자산                                                                                     
18                                            이연법인세자산                                                                                     
19                                            기타비유동자산                                                                                     
20                                               자산총계                                                                                     
21                                               유동부채                                                                                     
22                                               매입채무                                                                                     
23                                              단기차입금                                                                                     
24                                               미지급금                                                                                     
25                                                선수금                                                                                     
26                                                예수금                                                          

In [15]:
df_is.iloc[:, [1, 2, 6]]

[D310000] 손익계산서, 기능별 분류 - 연결 | Income statement, by function of expense - Consolidated financial statements (Unit: KRW)  \
                                                                                                                  label_ko   
0                                                영업수익                                                                        
1                                                매출원가                                                                        
2                                               매출총이익                                                                        
3                                             판매비와관리비                                                                        
4                                                영업이익                                                                        
5                                                기타이익                                                                        
6                                                기타손실                                                                        
7                                               지분법이익                                                                        
8                                                금융수익                                                                        
9                                                금융비용                                                                        
10                                    법인세비용차감전순이익(손실)                                                                        
11                                          법인세비용(수익)                                                                        
12                                         계속영업이익(손실)                                                                        
13                                          당기순이익(손실)                                                                        
14                         지배기업의 소유주에게 귀속되는 당기순이익(손실)                                                                        
15                              비지배지분에 귀속되는 당기순이익(손실)                                                                        
16                                         기본주당이익(손실)                                                                        
17                                         희석주당이익(손실)                                                                        

                                                         20240101-20240331  
                                             label_en            (연결재무제표,)  
0                                             Revenue 71,915,601,000,000.0  
1                                       Cost of sales 45,886,332,000,000.0  
2                                        Gross profit 26,029,269,000,000.0  
3                 Selling and administrative expenses 19,423,260,000,000.0  
4                             Operating profit (loss)  6,606,009,000,000.0  
5                                         Other gains    445,281,000,000.0  
6                                        Other losses    381,393,000,000.0  
7   Share of net profit of associates and joint ve...    214,833,000,000.0  
8                                    Financial income  3,484,504,000,000.0  
9                                   Financial expense  2,662,511,000,000.0  
10                           Profit before income tax  7,706,723,000,000.0  
11                       Income tax expense (benefit)    952,015,000,000.0  
12           Profit (loss) from continuing operations  6,754,708,000,000.0  
13                                Profit for the year  6,754,708,000,000.0  
14  Profit attributable to Owners of the parent co...  6,621,030,000,000.0  
15   Profit attributable to Non-controlling interests    133,678,000,000.0  
16                                            - Basic                975.0  
17                 

In [16]:
df_cf.iloc[:, [1, 2, 7]]

[D520000] 현금흐름표, 간접법 - 연결 | Statement of cash flows, indirect method - Consolidated financial statements (Unit: KRW)  \
                                                                                                               label_ko   
0                                            영업활동현금흐름                                                                     
1                                       영업에서 창출된 현금흐름                                                                     
2                                               당기순이익                                                                     
3                                                  조정                                                                     
4                                  영업활동으로 인한 자산부채의 변동                                                                     
5                                              이자의 수취                                                                     
6                                              이자의 지급                                                                     
7                                              배당금 수입                                                                     
8                                             법인세 납부액                                                                     
9                                            투자활동현금흐름                                                                     
10                                    단기금융상품의 순감소(증가)                                                                     
11                               단기상각후원가금융자산의 순감소(증가)                                                                     
12                           단기당기손익-공정가치금융자산의 순감소(증가)                                                                     
13                                         장기금융상품의 처분                                                                     
14                                         장기금융상품의 취득                                                                     
15                               기타포괄손익-공정가치측정금융자산의처분                                                                     
16                               기타포괄손익-공정가치측정금융자산의취득                                                                     
17                                  당기손익-공정가치금융자산의 처분                                                                     
18                                  당기손익-공정가치금융자산의 취득                                                                     
19                                 관계기업 및 공동기업 투자의 처분                                                                     
20                                 관계기업 및 공동기업 투자의 취득                                                                     
21                                           유형자산의 처분                                                                     
22                                           유형자산의 취득                                                                     
23                                           무형자산의 처분                                                                     
24                                           무형자산의 취득                                                                     
25                                    사업결합으로 인한 현금유출액                                                                     
26                      매각예정으로 분류된 비유동자산이나 처분자산집단의 처분                                                                     
27                                 기타투자활동으로 인한 현금유출입액                                                                     
28                                           재무활동현금흐름                                                                     
29                                     단기차입금의 순증가(감소)                                                                     
30                                          장기차입금의 차입              

# pykrx

In [391]:
date = last_business_day(2024, 7, 28).strftime("%Y%m%d")

In [386]:
data = pykrx.stock.get_market_fundamental(date, market="ALL")
com_data = data[data.index == "005930"]


In [390]:
com_data

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
005930,52002,37.96,1.56,2131,1.78,1444


In [392]:
data = pykrx.stock.get_market_ohlcv(date, date, "005930")

In [398]:
price = data["종가"].item()

In [56]:
financials_keys = ["net_profit_growth_from_last_quarter", "net_profit_growth_from_last_year",
                    "operating_profit_growth_from_last_quarter", "operating_profit_growth_from_last_year",
                    "revenue_profit_growth_from_last_quarter", "revenue_profit_growth_from_last_year"]
cashflow_keys = ["operating_cash_flow_from_last_quarter", "operating_cash_flow_from_last_year"]
info_keys = ["ROE", "PEGR", "PBR", "PSR"]